Sam Odle <br>
October 2, 2021  <br>
Project 2  <br>

In [3]:
import pandas as pd
pd.set_option("display.max_rows", None)

# Part 1: Dataset Download And Extraction

##### Load csv correctly into a DataFrame and show contents in a cell